In [0]:
!pip install gmplot

    100% |████████████████████████████████| 122kB 3.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/81/6a/76/4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot


In [0]:
import gmplot

import json

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import datetime as dt
import math
import numpy as np

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd "/content/drive/My Drive/Vilnius University/Vilnius City Crime Prediction"

/content/drive/My Drive/Vilnius University/Vilnius City Crime Prediction


In [0]:
from importlib.machinery import SourceFileLoader
common = SourceFileLoader('common', "Modules/common.py").load_module()

In [0]:
PARAMS = json.load(open('Params.json', 'r'))

In [0]:
def VisualizeCrimeOnMap(positionFrom, x, y, isHeat,crimes, fileName):
  pos1 = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], x, 0))
  posTo = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], x, y,))
  pos2 = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], 0, y,))
  gmap = gmplot.GoogleMapPlotter((positionFrom[0] + posTo[0]) / 2.0, (positionFrom[1] + posTo[1]) / 2.0, 14)
  gmap.apikey = PARAMS["GoogleApiKey"]
  if isHeat:
    gmap.heatmap(crimes.Lat, crimes.Long, threshold=15, radius=30, gradient=None, opacity=0.8, dissipating=True)
  else:
    gmap.scatter(crimes.Lat, crimes.Long, '#3B0B39', size=10, marker=False)
  gmap.plot([positionFrom[0], pos1[0], posTo[0], pos2[0], positionFrom[0]], [positionFrom[1], pos1[1], posTo[1], pos2[1], positionFrom[1]],  'cornflowerblue', edge_width=2)
  gmap.draw("Data/" + fileName)

crimes = pd.read_csv("Data/crimes.csv", delimiter="\t", index_col=0)

crimes = crimes[(crimes.Date >= "2018-01-01") & (crimes.Date < '2019-01-01')]
print(len(crimes.index))
print(crimes.head())
VisualizeCrimeOnMap(PARAMS["StartPosition"], PARAMS["TotalSize"][0], PARAMS["TotalSize"][1], True, crimes, "crimeMap2018.html")

772
            Date       ID        Lat      Long      Type
1491  2018-01-01  5730613  54.705766  25.27643  10020202
1492  2018-01-01  5793224  54.705766  25.27643  10020202
1493  2018-01-01  5795070  54.705766  25.27643  10020202
1494  2018-01-01  5825325  54.705766  25.27643  10020202
1495  2018-01-01  5825327  54.705766  25.27643  10020202


In [0]:
def SplitIntoGrid(crimes, startPosition, totalSize, gridSize): #TODO create function test
  cells = []
  for y in range(int(math.floor(totalSize[1] / gridSize[1]))):
    for x in range(int(math.floor(totalSize[0] / gridSize[0]))):
      #print("{} {}".format(x, y))
      cellStart = tuple(common.OffsetCoordinate(startPosition[0], startPosition[1], x * gridSize[0], y * gridSize[1],))
      cellEnd = tuple(common.OffsetCoordinate(cellStart[0], cellStart[1], gridSize[0], gridSize[1],))
      cells.append(crimes[(crimes.Lat > cellStart[0]) & (crimes.Lat < cellEnd[0]) & (crimes.Long > cellStart[1]) & (crimes.Long < cellEnd[1])])
  return cells

In [0]:
def SplitIntoPeriods(crimes, startDate, endDate, periodLenght): #TODO create function test
  periods = []
  DATE_FORMAT = "%Y-%m-%d"
  periodStart = dt.datetime.strptime(startDate, DATE_FORMAT)
  periodEnd = periodStart + dt.timedelta(days=periodLenght)
  i = 0
  while periodStart <= (dt.datetime.strptime(endDate, DATE_FORMAT)):
    lastDayNotIncluded = periodEnd - dt.timedelta(days=1)
    periods.append(crimes[(crimes.Date >= periodStart.strftime(DATE_FORMAT)) & (crimes.Date <= lastDayNotIncluded.strftime(DATE_FORMAT))])
    print("{} {} {}".format(i, periodStart.strftime("%Y-%m-%d"), lastDayNotIncluded.strftime("%Y-%m-%d")))
    i = i + 1
    periodStart = periodStart + dt.timedelta(days=periodLenght)
    periodEnd = periodStart + dt.timedelta(days=periodLenght)
  return periods

In [0]:
def VisualizeCrimeGrid(positionFrom, crimes, crimesInGrid, totalSize, gridSize, fileName): #TODO create function test
  pos1 = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], totalSize[0], 0)) #TODO move the square coords crations to common
  posTo = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], totalSize[0], totalSize[1],))
  pos2 = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], 0, totalSize[1],))
  gmap = gmplot.GoogleMapPlotter((positionFrom[0] + posTo[0]) / 2.0, (positionFrom[1] + posTo[1]) / 2.0, 14)
  gmap.apikey = PARAMS["GoogleApiKey"]
  gmap.scatter(crimes.Lat, crimes.Long, '#3B0B39', size=10, marker=False)
  gmap.plot([positionFrom[0], pos1[0], posTo[0], pos2[0], positionFrom[0]], [positionFrom[1], pos1[1], posTo[1], pos2[1], positionFrom[1]],  'cornflowerblue', edge_width=2)
  
  for i in range(len(crimesInGrid)):
    y, x = divmod(i, int(totalSize[0]/gridSize[0]))
    temp1 = tuple(common.OffsetCoordinate(positionFrom[0], positionFrom[1], x * gridSize[0], y * gridSize[1]))
    temp2 = tuple(common.OffsetCoordinate(temp1[0], temp1[1], gridSize[0], 0,))
    temp3 = tuple(common.OffsetCoordinate(temp1[0], temp1[1], gridSize[0], gridSize[1],))
    temp4 = tuple(common.OffsetCoordinate(temp1[0], temp1[1], 0, gridSize[1],))
    if crimesInGrid[i] > 0:
      gmap.polygon([temp1[0], temp2[0], temp3[0], temp4[0], temp1[0]], [temp1[1], temp2[1], temp3[1], temp4[1], temp1[1]],  'cornflowerblue')
    else:
      gmap.plot([temp1[0], temp2[0], temp3[0], temp4[0], temp1[0]], [temp1[1], temp2[1], temp3[1], temp4[1], temp1[1]],  'cornflowerblue', edge_width=2)
  
  gmap.draw("Data/" + fileName)

In [0]:
crimes = pd.read_csv("Data/crimes.csv", delimiter="\t", index_col=0)

crimeGridInPeriods = []
periods = SplitIntoPeriods(crimes, PARAMS["StartDate"], PARAMS["EndDate"], PARAMS["Period"])
for period in periods:
  cells = SplitIntoGrid(period, tuple(PARAMS["StartPosition"]), tuple(PARAMS["TotalSize"]), tuple(PARAMS["GridSize"]))
  crimeGrid = [np.sign(len(c.index)) for c in cells]
  crimeGridInPeriods.append(crimeGrid)
 
crimeGridInPeriodsDf = pd.DataFrame(crimeGridInPeriods)
print(crimeGridInPeriodsDf.tail())
crimeGridInPeriodsDf.to_csv("Data/dataForModel.csv", sep="\t")
  
VisualizeCrimeGrid(tuple(PARAMS["StartPosition"]), periods[2], crimeGridInPeriods[2], tuple(PARAMS["TotalSize"]), tuple(PARAMS["GridSize"]), "crimesInPeriods.html")
  
#periods = SplitIntoPeriods(crimes, START_DATE, END_DATE, PERIOD)
#VisualizeCrimeOnMap(START_POSITION, TOTAL_X, TOTAL_Y, False, periods[0], "crimesInPeriods.html")

#cells = SplitIntoGrid(crimes, START_POSITION, TOTAL_SIZE, GRID_SIZE)
#VisualizeCrimeOnMap(START_POSITION, TOTAL_X, TOTAL_Y, False, cells[11*12 + 11], "crimesInGrid.html")

0 2015-01-01 2015-01-01
1 2015-01-02 2015-01-02
2 2015-01-03 2015-01-03
3 2015-01-04 2015-01-04
4 2015-01-05 2015-01-05
5 2015-01-06 2015-01-06
6 2015-01-07 2015-01-07
7 2015-01-08 2015-01-08
8 2015-01-09 2015-01-09
9 2015-01-10 2015-01-10
10 2015-01-11 2015-01-11
11 2015-01-12 2015-01-12
12 2015-01-13 2015-01-13
13 2015-01-14 2015-01-14
14 2015-01-15 2015-01-15
15 2015-01-16 2015-01-16
16 2015-01-17 2015-01-17
17 2015-01-18 2015-01-18
18 2015-01-19 2015-01-19
19 2015-01-20 2015-01-20
20 2015-01-21 2015-01-21
21 2015-01-22 2015-01-22
22 2015-01-23 2015-01-23
23 2015-01-24 2015-01-24
24 2015-01-25 2015-01-25
25 2015-01-26 2015-01-26
26 2015-01-27 2015-01-27
27 2015-01-28 2015-01-28
28 2015-01-29 2015-01-29
29 2015-01-30 2015-01-30
30 2015-01-31 2015-01-31
31 2015-02-01 2015-02-01
32 2015-02-02 2015-02-02
33 2015-02-03 2015-02-03
34 2015-02-04 2015-02-04
35 2015-02-05 2015-02-05
36 2015-02-06 2015-02-06
37 2015-02-07 2015-02-07
38 2015-02-08 2015-02-08
39 2015-02-09 2015-02-09
40 2015-02

In [0]:
places = pd.read_csv("Data/places.csv", delimiter="\t", index_col=0)

#print(places.Long)
#places = places[places.Types.str.strip("['").str.startswith("park")]
places = SplitIntoGrid(places, PARAMS["StartPosition"], PARAMS["TotalSize"], PARAMS["GridSize"])
temp = places[1]
print(len(temp[temp.Types.str.strip("[']").str.startswith("park")].index))
print(temp)
VisualizeCrimeOnMap(PARAMS["StartPosition"], PARAMS["TotalSize"][0], PARAMS["TotalSize"][1], False, temp, "placesMap.html")
#places.tail()

1
                                           ID        Lat       Long  \
51   3031f4f188ea17c257fa6238a7695affd5b37b3c  54.671228  25.269203   
95   59eb16b2ed46ef889578922295b4dd349867d94e  54.672743  25.266562   
149  9deff5b8a8ec30a2d093d25d4d68f7b51777e5bb  54.671542  25.268462   
187  0bf4fb84ab6e511b9b0cc1b6261a22148b0494be  54.673230  25.267623   
244  e57146c676f704f791efb984415aca63f53eaea2  54.670921  25.259787   
374  f739c2c48af55322f8babeb0521a8b81c0f5f8fc  54.671865  25.269345   

                                                 Types  
51   ['restaurant', 'point_of_interest', 'food', 'e...  
95     ['store', 'point_of_interest', 'establishment']  
149  ['night_club', 'bar', 'point_of_interest', 'es...  
187     ['park', 'point_of_interest', 'establishment']  
244  ['bank', 'finance', 'point_of_interest', 'esta...  
374  ['transit_station', 'point_of_interest', 'esta...  


In [0]:
crimes = pd.read_csv("Data/crimes.csv", delimiter="\t", index_col=0)
places = pd.read_csv("Data/places.csv", delimiter="\t", index_col=0)

placesInGrid = SplitIntoGrid(places, PARAMS["StartPosition"], PARAMS["TotalSize"], PARAMS["GridSize"])
crimeGridInPeriods = []
periods = SplitIntoPeriods(crimes, PARAMS["StartDate"], PARAMS["EndDate"], PARAMS["Period"])
for period in periods:
  cells = SplitIntoGrid(period, tuple(PARAMS["StartPosition"]), tuple(PARAMS["TotalSize"]), tuple(PARAMS["GridSize"]))
  crimeGrid = []
  for i in range(len(cells)):
    crimeGrid.append(np.sign(len(cells[i].index)))
    
    temp = placesInGrid[i] #TODO this can be optimized by calculating before the loop
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("bar")].index))
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("restaurant")].index))
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("store")].index))
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("night_club")].index))
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("park")].index))
    crimeGrid.append(len(temp[temp.Types.str.strip("[']").str.startswith("bank")].index))
    
    stationsCount = len(temp[temp.Types.str.strip("[']").str.startswith("bus_station")].index) + len(temp[temp.Types.str.strip("[']").str.startswith("train_station")].index) + len(temp[temp.Types.str.strip("[']").str.startswith("transit_station")].index)
    crimeGrid.append(stationsCount)
  crimeGridInPeriods.append(crimeGrid)
 
crimeGridInPeriodsDf = pd.DataFrame(crimeGridInPeriods)
print(crimeGridInPeriodsDf.tail())
crimeGridInPeriodsDf.to_csv("Data/dataForModel+.csv", sep="\t")

0 2015-01-01 2015-01-01
1 2015-01-02 2015-01-02
2 2015-01-03 2015-01-03
3 2015-01-04 2015-01-04
4 2015-01-05 2015-01-05
5 2015-01-06 2015-01-06
6 2015-01-07 2015-01-07
7 2015-01-08 2015-01-08
8 2015-01-09 2015-01-09
9 2015-01-10 2015-01-10
10 2015-01-11 2015-01-11
11 2015-01-12 2015-01-12
12 2015-01-13 2015-01-13
13 2015-01-14 2015-01-14
14 2015-01-15 2015-01-15
15 2015-01-16 2015-01-16
16 2015-01-17 2015-01-17
17 2015-01-18 2015-01-18
18 2015-01-19 2015-01-19
19 2015-01-20 2015-01-20
20 2015-01-21 2015-01-21
21 2015-01-22 2015-01-22
22 2015-01-23 2015-01-23
23 2015-01-24 2015-01-24
24 2015-01-25 2015-01-25
25 2015-01-26 2015-01-26
26 2015-01-27 2015-01-27
27 2015-01-28 2015-01-28
28 2015-01-29 2015-01-29
29 2015-01-30 2015-01-30
30 2015-01-31 2015-01-31
31 2015-02-01 2015-02-01
32 2015-02-02 2015-02-02
33 2015-02-03 2015-02-03
34 2015-02-04 2015-02-04
35 2015-02-05 2015-02-05
36 2015-02-06 2015-02-06
37 2015-02-07 2015-02-07
38 2015-02-08 2015-02-08
39 2015-02-09 2015-02-09
40 2015-02